In [1]:
from glob import glob

from collections import defaultdict
import json
import os

In [2]:
questions = defaultdict(set)
q = []
for f in glob('base-questions-kerajaan-*/*'):
    b = os.path.split(f)[0]
    with open(f) as fopen:
        data = json.load(fopen)
    data = '\n'.join(data)
    data = data.split('\n\n')
    if len(data) == 1:
        data = data[0].split('\n')
    else:
        datas = []
        for d in data:
            splitted = d.split('\n ')
            splitted = [s.strip() for s in splitted]
            datas.extend(splitted)
        data = datas
    data = ['.'.join(d.split('.')[1:]).strip() for d in data]
    data = [d for d in data if len(d) > 3]
        
    data = set(data)
    questions[b] |= data

In [3]:
total_questions = 0
all_questions = []
for k, v in questions.items():
    total_questions += len(v)
    all_questions.extend(v)
    
total_questions

31291

In [4]:
all_questions = sorted(all_questions)

In [6]:
all_questions[0]

'Adakah Agrobank memberi tumpuan khas kepada pembiayaan projek-projek yang berkaitan dengan pembangunan infrastruktur di luar bandar di Malaysia?'

In [17]:
texts = []
for i in range(len(all_questions)):
    filename = f'answer-question-chatgpt3-gov.my/{i}.json'
    if not os.path.exists(filename):
        continue
    
    with open(filename) as fopen:
        d = json.load(fopen)
    
    try:
    
        d = '\n'.join(d)
    except:
        continue
    
    texts.append(d.strip())
    texts.append(all_questions[i].strip())

In [20]:
texts = list(set(texts))
len(texts)

62456

In [21]:
with open('answer-random-questions-gov.my.texts', 'w') as fopen:
    for t in set(texts):
        fopen.write(f'{json.dumps(t)}\n')

In [22]:
!cp answer-random-questions-gov.my.texts ../ctranslate2

In [23]:
mapping = {}
with open('../ctranslate2/answer-random-questions-gov.my.texts.requested') as fopen:
    for l in fopen:
        l = json.loads(l)
        mapping[l['src']] = l['r']
len(mapping)

62456

In [25]:
qa, texts = [], []
for i in range(len(all_questions)):
    q = all_questions[i]
    
    filename = f'answer-question-chatgpt3-gov.my/{i}.json'
    if not os.path.exists(filename):
        continue
    with open(filename) as fopen:
        answer = json.load(fopen)
    
    if answer is None:
        continue
        
    answer = '\n'.join(answer)
    
    qa.append({
        'question': q.strip(),
        'answer': answer.strip()
    })
    
    texts.extend([q.strip(), answer.strip()])
    
len(qa)

31236

In [26]:
with open('malaysian-general-qa-gov-my.jsonl', 'w') as fopen:
    for q in qa:
        q['question_ms'] = mapping.get(q['question'])
        q['answer_ms'] = mapping.get(q['answer'])
        fopen.write(f'{json.dumps(q)}\n')

In [27]:
q

{'question': 'What is the role of the National Service Training Department (JLKN) in developing youth leadership in Malaysia?\n2. How does JLKN provide quality training and national service programs for Malaysian youth?\n3. What sets JLKN training programs apart from other national service programs in Malaysia?\n4. What is JLKN\'s contribution in fostering patriotism among Malaysian youth?\n5. What do Malaysian youth understand about "national identity" through their experiences in JLKN programs?\n6. What is JLKN\'s role in involving Malaysian youth in community service activities?\n7. How does JLKN raise awareness among Malaysian youth about current social and environmental issues?\n8. What can Malaysian youth explain about the importance of national service in national development?\n9. How does JLKN help Malaysian youth learn moral values and ethics through training and national service?\n10. What are the positive effects of JLKN programs on the personal development of Malaysian yout

In [28]:
!wc -l malaysian-general-qa-gov-my.jsonl

31236 malaysian-general-qa-gov-my.jsonl


In [29]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='malaysian-general-qa-gov-my.jsonl',
    path_in_repo='malaysian-general-qa-gov-my.jsonl',
    repo_id='mesolitica/chatgpt-malaysian-general-qa',
    repo_type='dataset',
)

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


malaysian-general-qa-gov-my.jsonl:   0%|          | 0.00/108M [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/chatgpt-malaysian-general-qa/blob/main/malaysian-general-qa-gov-my.jsonl'